In [31]:
from dotenv import load_dotenv
import requests
import os
from math import exp
from twilio.rest import Client

In [26]:
def get_nba_scores():
    url = "https://site.api.espn.com/apis/site/v2/sports/basketball/nba/scoreboard"
    response = requests.get(url)
    data = response.json()

    games = []
    for event in data['events']:
        competition = event['competitions'][0]
        home = competition['competitors'][0]
        away = competition['competitors'][1]
        games.append({
            'home_team': home['team']['displayName'],
            'home_score': home['score'],
            'away_team': away['team']['displayName'],
            'away_score': away['score'],
            'status': competition['status']['type']['description']
        })

    return games

# Example usage
for game in get_nba_scores():
    print(game)


{'home_team': 'Indiana Pacers', 'home_score': '0', 'away_team': 'New York Knicks', 'away_score': '0', 'status': 'Scheduled'}


In [27]:
# really simple win probability func
def win_prob(score_diff, time_remaining_seconds):
    """
    Simple model: win prob for team leading.
    """
    if time_remaining_seconds <= 0:
        return 1.0 if score_diff > 0 else 0.0

    k = 0.045  # steepness factor
    midpoint = 0  # centered @ score_diff = 0

    # Add weight for time remaining
    time_factor = 1 - (time_remaining_seconds / 2880)  # 48 mins => 2880 sec
    adjusted_diff = score_diff * time_factor

    prob = 1 / (1 + exp(-k * (adjusted_diff - midpoint)))
    return round(prob, 3)


In [28]:
# Example: 4 pt lead, 300 sec (5 mins left), calculate win prodb
win_prob(4, 300)

0.54

In [29]:
def parse_game_time(period, clock):
    # period: 1–4 for quarters, 5+ for OT
    # clock: mm:ss
    minutes, seconds = map(int, clock.split(":"))
    elapsed = (period - 1) * 720 + (60 - minutes) * 60 + (60 - seconds)  # crude
    return max(0, 2880 - elapsed)  # total regulation = 2880 seconds


In [30]:
FAVORITE_TEAM = "Indiana Pacers"  # Customize this
def monitor_games_and_alert():
    games = get_nba_scores()
    for game in games:
        if game['status'] != "In Progress":
            continue

        # Determine if favorite team is playing
        if FAVORITE_TEAM in [game['home_team'], game['away_team']]:
            score_diff = int(game['home_score']) - int(game['away_score'])
            if game['away_team'] == FAVORITE_TEAM:
                score_diff = -score_diff

            competition = game['status']
            clock = game.get("clock", "0:00")
            period = int(game.get("period", {}).get("number", 1))
            time_left = parse_game_time(period, clock)

            prob = win_prob(score_diff, time_left)
            losing_prob = 1 - prob

            if losing_prob > 0.8:
                send_text_alert(FAVORITE_TEAM, losing_prob)


In [22]:
def send_text_alert(team, prob):
    account_sid = os.getenv('ACCOUNT_SID')
    auth_token = os.getenv('AUTH_TOKEN')
    from_phone = os.getenv('FROM_PHONE')  # Twilio number
    to_phone = os.getenv('TO_PHONE')  # Your phone

    client = Client(account_sid, auth_token)
    message = client.messages.create(
        body=f"ALERT: {team} has an {round(prob*100)}% chance of losing!",
        from_=from_phone,
        to=to_phone
    )
    print("Alert sent:", message.sid)

In [32]:
send_text_alert("Pacers", 2)

Alert sent: SM5cae130ecc9fa418b35161c9b3c5b61f
